In [2]:
import pandas as pd
import preprocessing

In [3]:
test_data = pd.read_csv('./data/filled_test_set.csv')
print(test_data.describe())
nan_data_info = pd.DataFrame({'NaN Count': test_data.isna().sum(), 'Data Type': test_data.dtypes})
print(nan_data_info)

            srch_id       site_id  visitor_location_country_id  \
count  4.959183e+06  4.959183e+06                 4.959183e+06   
mean   1.666460e+05  9.979214e+00                 1.752504e+02   
std    9.614992e+04  7.667722e+00                 6.590284e+01   
min    1.000000e+00  1.000000e+00                 1.000000e+00   
25%    8.331150e+04  5.000000e+00                 1.000000e+02   
50%    1.670950e+05  5.000000e+00                 2.190000e+02   
75%    2.499660e+05  1.400000e+01                 2.190000e+02   
max    3.327870e+05  3.400000e+01                 2.310000e+02   

       visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  \
count             4.959183e+06          4.959183e+06     4.959183e+06   
mean              3.374933e+00          1.771507e+02     1.738371e+02   
std               1.564156e-01          2.483270e+01     6.835006e+01   
min               1.000000e+00          0.000000e+00     1.000000e+00   
25%               3.374933e+00          

In [7]:
import joblib
test_set = test_data.drop(columns=['prop_id', 'site_id', 'random_bool', 'random_bool', 'srch_destination_id', 'visitor_location_country_id'])
model = joblib.load('models/lightgbm.pkl')

In [10]:
qids = test_set.groupby("srch_id")["srch_id"].count().to_numpy()
X = test_set.drop(["srch_id"], axis=1)
print(X)

         visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  \
0                       3.374933             177.15073            219.0   
1                       3.374933             177.15073            219.0   
2                       3.374933             177.15073            219.0   
3                       3.374933             177.15073            219.0   
4                       3.374933             177.15073            219.0   
...                          ...                   ...              ...   
4959178                 3.374933             177.15073            117.0   
4959179                 3.374933             177.15073            117.0   
4959180                 3.374933             177.15073            117.0   
4959181                 3.374933             177.15073            117.0   
4959182                 3.374933             177.15073            117.0   

         prop_starrating  prop_review_score  prop_brand_bool  \
0                    3.0           

In [11]:
output = model.predict(X)
print(output)

[ 0.08375423  0.03336948 -0.66657142 ... -0.40751584 -0.55832326
  0.20740629]


In [12]:
result = test_data[['srch_id', 'prop_id']]
result['pred'] = output
print(result)

          srch_id  prop_id      pred
0             1.0   3180.0  0.083754
1             1.0   5543.0  0.033369
2             1.0  14142.0 -0.666571
3             1.0  22393.0 -0.450701
4             1.0  24194.0  0.873681
...           ...      ...       ...
4959178  332787.0  32019.0  0.012922
4959179  332787.0  33959.0  1.491002
4959180  332787.0  35240.0 -0.407516
4959181  332787.0  94437.0 -0.558323
4959182  332787.0  99509.0  0.207406

[4959183 rows x 3 columns]


C:\Users\china\AppData\Local\Temp\ipykernel_3356\4200473951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['pred'] = output


In [13]:
sorted_df = result.groupby('srch_id').apply(lambda x: x.sort_values(by='pred', ascending=False))
sorted_df.reset_index(drop=True, inplace=True)

                   srch_id  prop_id      pred
srch_id                                      
1.0      23            1.0  99484.0  1.402691
         9             1.0  54937.0  1.351685
         12            1.0  61934.0  1.254387
         5             1.0  28181.0  1.004039
         4             1.0  24194.0  0.873681
...                    ...      ...       ...
332787.0 4959177  332787.0  29018.0  0.263190
         4959182  332787.0  99509.0  0.207406
         4959178  332787.0  32019.0  0.012922
         4959180  332787.0  35240.0 -0.407516
         4959181  332787.0  94437.0 -0.558323

[4959183 rows x 3 columns]


C:\Users\china\AppData\Local\Temp\ipykernel_3356\1858461019.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = result.groupby('srch_id').apply(lambda x: x.sort_values(by='pred', ascending=False))


In [15]:
print(sorted_df)

    srch_id   prop_id      pred
29      3.0  131940.0  2.068683
30      3.0   63039.0  1.130917
31      3.0   46861.0  0.346563
32      3.0   43133.0  0.268612
33      3.0   79176.0  0.111417
34      3.0   27753.0  0.061322
35      3.0   11425.0 -0.040811
36      3.0    6533.0 -0.323126
37      3.0   47377.0 -0.340674
38      3.0   91337.0 -0.409803
39      3.0  131968.0 -0.428518
40      3.0  132637.0 -0.581521
41      3.0   96614.0 -0.592091
42      3.0   80734.0 -0.628449
43      3.0   43362.0 -0.636740
44      3.0   24915.0 -0.653194
45      3.0   32382.0 -0.679123
46      3.0   47238.0 -1.010026
47      3.0   90968.0 -1.096190
48      3.0  119120.0 -1.118413
49      3.0   22325.0 -1.147680
50      3.0   71543.0 -1.174972
51      3.0   64168.0 -1.185673
52      3.0  124819.0 -1.188311
53      3.0   45550.0 -1.255730
54      3.0   93176.0 -1.315099
55      3.0  131397.0 -1.726382
56      6.0   24695.0  0.979521
57      6.0   31470.0 -0.237506
58      6.0  139558.0 -0.789322
59      

In [18]:
final_res = sorted_df[['srch_id', 'prop_id']]
final_res = final_res.astype(int)
print(final_res)
final_res.to_csv('./data/final_res.csv', index=False)

         srch_id  prop_id
0              1    99484
1              1    54937
2              1    61934
3              1    28181
4              1    24194
...          ...      ...
4959178   332787    29018
4959179   332787    99509
4959180   332787    32019
4959181   332787    35240
4959182   332787    94437

[4959183 rows x 2 columns]
